In [1]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import os

In [2]:
def get_video_files(directory):
    video_files = []
    subfolders = ['deceptive', 'truthful']
    for subfolder in subfolders:
        path = os.path.join(directory, subfolder)
        files = [os.path.join(subfolder, f) for f in os.listdir(path) if f.endswith('_metrics.csv')]
        video_files.extend(files)
    return video_files

In [33]:
def load_data_from_files(file_list, directory):
    data_frames = []
    for file in file_list:
        path = os.path.join(directory, file)
        df = pd.read_csv(path)
        pd.get_dummies(df['Emotion']) # Must convert to one-hot encoding here to avoid issues with missing values/columns
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [34]:
video_metric_directory = '../video_metrics/'
video_files = get_video_files(video_metric_directory)
train_files, test_files = train_test_split(video_files, test_size=0.2, random_state=42)

train_data = load_data_from_files(train_files, video_metric_directory)
test_data = load_data_from_files(test_files, video_metric_directory)

C:\Users\cpfit\AppData\Local\Temp\ipykernel_32848\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)
C:\Users\cpfit\AppData\Local\Temp\ipykernel_32848\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)


In [35]:
train_data.drop(columns=['Video_Path'], inplace=True)
test_data.drop(columns=['Video_Path'], inplace=True)

train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())

C:\Users\cpfit\AppData\Local\Temp\ipykernel_32848\2728043661.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data = train_data.fillna(train_data.median())
C:\Users\cpfit\AppData\Local\Temp\ipykernel_32848\2728043661.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data = test_data.fillna(test_data.median())


In [36]:
X_train = train_data.drop(columns=['Label'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['Label'])
y_test = test_data['Label']

# XGBoost

In [37]:
# Initialize and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [48]:
# Define the parameters to search using linspace for continuous ranges
params = {
    'max_depth': np.arange(3, 20),
    'gamma': np.linspace(0.1, 5, 20),
    'subsample': np.linspace(0.5, 1.0, 11),
    'colsample_bytree': np.linspace(0.5, 1.0, 11),
    'n_estimators': np.linspace(100, 1000, 10).astype(int),
    'learning_rate': np.linspace(0.01, 0.5, 20)
}

# Setup the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=xgb_model, param_distributions=params, n_iter=10, scoring='accuracy', cv=4, verbose=2,  n_jobs=7)

In [49]:
# Fit the random search model
random_cv.fit(X_train, y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
       0.13894737, 0.16473684, 0.19052632, 0.21631579, 0.24210526,
       0.26789474, 0.29368421, 0.31947368, 0.34526316, 0.37105263,
       0.39684211, 0.42263158, 0.44842105, 0.47421053, 0.5       ]),
                                        'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                                        'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000]),
                                        'subsample': array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])},
                   scoring='accuracy', verbose=2)

In [50]:
# Best parameters and score
print("Best parameters:", random_cv.best_params_)
print("Best accuracy:", random_cv.best_score_)

# Evaluate on the test set
print("Test set accuracy:", random_cv.score(X_test, y_test))

# Best parameters: {'subsample': 0.95, 'n_estimators': 900, 'max_depth': 10, 'learning_rate': 0.3452631578947368, 'gamma': 4.742105263157895, 'colsample_bytree': 0.6}
# Best accuracy: 0.617920060158935
# Test set accuracy: 0.5191781529089876

Best parameters: {'subsample': 0.95, 'n_estimators': 900, 'max_depth': 10, 'learning_rate': 0.3452631578947368, 'gamma': 4.742105263157895, 'colsample_bytree': 0.6}
Best accuracy: 0.617920060158935
Test set accuracy: 0.5191781529089876


In [51]:
# Evaluate the best model on the test set
best_model = random_cv.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Best accuracy:", random_cv.best_score_)
print("Test set accuracy:", accuracy)
print("Classification Report:\n", class_report)

Best accuracy: 0.617920060158935
Test set accuracy: 0.5191781529089876
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.31      0.40     10063
           1       0.50      0.74      0.60      9308

    accuracy                           0.52     19371
   macro avg       0.53      0.53      0.50     19371
weighted avg       0.53      0.52      0.50     19371



In [17]:
import pickle
best_model = random_cv.best_estimator_

filename = 'finalized_xgb_classifier.pkl'
pickle.dump(best_model, open(filename, 'wb'))
